### Day 15

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

In [2]:
day_i = 15

In [3]:
%run start_day.py $day_i

Initializing day 15


In [4]:
cd /home/vincent/Documents/AdventOfCode/2021

/home/vincent/Documents/AdventOfCode/2021


In [5]:
PATH = f"day{day_i}/input{day_i}"

In [6]:
!wc -l $PATH

100 day15/input15


In [7]:
!head $PATH

1797218461142179319573884891481221191657631838831754357117998319396235714788116521813149318951532839
8696981297918411971289919865719836144693333162529142812974198292932295491956499215586135482941739392
2991519621128967692174919369871962711711888349621119249111241263371998761114982461871811915749818981
9519298621148891919219116926535193934326629926112136839413932239182391497985687943399987482691961448
3322441212551229339949332249462272248555173795333563937398919128799491931719178641531998141722584429
9118353711988171611983883569964392454546273683762581127881992188984158689612224523966629229869841368
6619293351892345695919931961961959917575811995111642219592114857133566312236841811951318937396921471
4119719294958136935585131572211882179799139693669968792713194888628799229972719918777119331922298192
2897361395359964817739628715924942441921486256913485426413911886166322118384197271142719438976157178
3389638941186823278892211919997494993715555987282539661565969785971121324112299762618146712

In [8]:
!tail $PATH

1919894989221529553399133386169999552895185552222755159922552431149786511973745343931818539315311175
2683381835121351182998489919911372894291894973399493194281379971969183416974346479151274291799853129
1751991384249781899947665921219827616999239519455929713722239711149225383616519712941889322712828418
3295919911592497192118828258118529133121427523281991929691965274368825689299671219749189813124887278
7828319259851721362716353874221716695129721491254728715913967965694595858939949833991856557249153792
4643162283932891824519919817417814174343124346663682549387955944171239182344299415172323771718859129
9382811937823973227813927835213688211415836968191527337792159834142739892661818877429739395149421471
3279512226121999191142499711269119643819431895822859387584439741423768775114929691745825321781118144
9391973246579999241217437296325675163824334513949186691912595622177779849572792973716569959311526846
4671438158382646393847419899596676468374198874175298772399482512793928838556984113933878117

In [9]:
def parse_input(inp):
    return [list(map(int, x)) for x in inp]

In [10]:
# real input
with open(PATH, 'r') as f:
    l = parse_input([x.strip() for x in f.readlines()])


In [11]:
# test input
test_str1 = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

l_1 = parse_input(test_str1.split('\n'))

In [12]:
l_1

[[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
 [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
 [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
 [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
 [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
 [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
 [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
 [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
 [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
 [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]]

In [77]:
import networkx as nx

def explore_graph(l, matrix, i=0, j=0, path_weight=0):
    if matrix[i,j] == 0 or matrix[i,j] > path_weight:
        matrix[i, j] = path_weight
    neighbors = [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
    neighbors = [x for x in neighbors if -1 not in x and x[0] != len(l) and x[1] != len(l[0])] # control edges
    for ni, nj in neighbors:
        new_path_weight = path_weight + l[ni][nj]
        if matrix[ni, nj] == 0 or matrix[ni, nj] > new_path_weight:
            explore_graph(l, matrix, ni, nj, new_path_weight)

def compute_graph(l):
    n, m = len(l), len(l[0])
    G = nx.DiGraph()
    for i in range(n):
        for j in range(m):
            G.add_node((i, j))
    for i,j in G.nodes():
        neighbors = [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
        neighbors = [x for x in neighbors if -1 not in x and x[0] != len(l) and x[1] != len(l[0])] # control edges
        for ni, nj in neighbors:
            G.add_edge((i,j), (ni, nj), weight=l[ni][nj])
    return G

def modified_weight(l, i, j):
    n, m = len(l), len(l[0])
    ni, nj = i // n, j // m
    return (l[i % n][j % m] + ni + nj - 1) % 9 + 1

def compute_big_graph(l):
    n, m = len(l), len(l[0])
    G = nx.DiGraph()
    for i in range(5 * n):
        for j in range(5 * m):
            G.add_node((i, j))
    for i, j in G.nodes():
        neighbors = [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
        neighbors = [x for x in neighbors if -1 not in x and x[0] != 5 * len(l) and x[1] != 5 * len(l[0])] # control edges
        for ni, nj in neighbors:
            G.add_edge((i,j), (ni, nj), weight=modified_weight(l, ni, nj))
    return G

def solve1(l):
    shortest_paths = np.zeros((len(l), len(l[0])), dtype=int)
    explore_graph(l, shortest_paths)
    return shortest_paths[-1, -1]

def solve1bis(l):
    G = compute_graph(l)
    print(len(G.nodes()))
    return nx.algorithms.shortest_paths.generic.shortest_path_length(G, 
                                                          source=(0, 0), 
                                                          target=(len(l)-1, len(l[0])-1), 
                                                          weight='weight')

def solve2(l):
    G = compute_big_graph(l)
    print(len(G.nodes()))
    return nx.algorithms.shortest_paths.generic.shortest_path_length(G, 
                                                          source=(0, 0), 
                                                          target=(5*len(l)-1, 5*len(l[0])-1), 
                                                          weight='weight')

In [51]:
solve1bis(l_1)

100


40

In [ ]:
import sys
sys.setrecursionlimit(40000)
solve1(l)

In [52]:
solve1bis(l)

10000


537

In [75]:
G_1 = compute_big_graph(l_1)

In [76]:
node = (49, 49)
for n in G_1.neighbors(node):
    print(node, n, G_1.get_edge_data(node, n))

(49, 49) (48, 49) {'weight': 1}
(49, 49) (49, 48) {'weight': 8}


In [78]:
solve2(l_1)

2500


315

In [79]:
solve2(l)

250000


2881